<a href="https://colab.research.google.com/github/paviayyala/AIML-Lab/blob/main/Bulk_Certificate_Generator_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧾 Bulk Certificate Generator (Excel → PNG + PDF)

This notebook generates **PNG + PDF certificates** from an **Excel (.xlsx)** list.

**Steps**
1. Install requirements
2. Upload your `students.xlsx` and `certificate_template.png` **(or use Google Drive path)**
3. Run the generator → get a `certificates.zip` to download

**Excel columns (case-insensitive):** `Name`, `Course`, `Date`, `CertificateID` (ID optional)

> Tip: Make sure your template PNG has white space in the center for the name & course text.

In [ ]:
#@title 1) Install packages
!pip -q install pillow pandas openpyxl

In [30]:
#@title 2A) Upload files from your computer (run this OR 2B)
from google.colab import files
import os

os.makedirs('input', exist_ok=True)
print("Please upload your Excel (.xlsx) and Template (.png)")
uploaded = files.upload()

excel_path = None
template_path = None
for name, data in uploaded.items():
    out_path = os.path.join('input', name)
    with open(out_path, 'wb') as f:
        f.write(data)
    if name.lower().endswith('.xlsx'):
        excel_path = out_path
    if name.lower().endswith('.png'):
        template_path = out_path

print('Excel:', excel_path)
print('Template:', template_path)
assert excel_path, "No .xlsx uploaded."
assert template_path, "No .png uploaded."

Please upload your Excel (.xlsx) and Template (.png)


Saving Pavithran Ayyala.png to Pavithran Ayyala (4).png
Saving students_sample.xlsx to students_sample (6).xlsx
Excel: input/students_sample (6).xlsx
Template: input/Pavithran Ayyala (4).png


In [ ]:
#@title 2B) (Optional) Use Google Drive paths instead of upload
#@markdown Run this cell if your files are already in Drive. Then set the two paths below.
use_drive = False  #@param {type:"boolean"}
drive_excel_path = "/content/drive/MyDrive/certificates/students.xlsx"  #@param {type:"string"}
drive_template_path = "/content/drive/MyDrive/certificates/template.png"  #@param {type:"string"}

if use_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    excel_path = drive_excel_path
    template_path = drive_template_path
    print('Using Drive paths:')
    print('Excel:', excel_path)
    print('Template:', template_path)

In [31]:
#@title 3) Helper functions (font + drawing)
from PIL import Image, ImageDraw, ImageFont
import textwrap, os

def load_font(size):
    candidates = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
        "/usr/share/fonts/truetype/dejavu/DejaVuSerif-Bold.ttf",
        "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                pass
    return ImageFont.load_default()

def draw_certificate(template_path, name, course, date_str, cert_id):
    im = Image.open(template_path).convert("RGB")
    draw = ImageDraw.Draw(im)
    W, H = im.size

    title_font = load_font(96)
    name_font = load_font(86)
    body_font = load_font(48)
    small_font = load_font(36)

    # Title & subtitle
    #title = "Certificate of Completion"
    title = "Certificate of Participation"
    # Replaced textsize with textbbox
    tw, th = draw.textbbox((0,0), title, font=title_font)[2:]
    draw.text(((W - tw) / 2, 180), title, fill=(20,20,20), font=title_font)

    subtitle = "This certifies that"
    # Replaced textsize with textbbox
    sw, sh = draw.textbbox((0,0), subtitle, font=body_font)[2:]
    draw.text(((W - sw) / 2, 320), subtitle, fill=(75,75,75), font=body_font)

    # Name (auto-shrink if too long)
    n_font = name_font
    n_text = name.strip()
    max_width = int(W*0.8)
    # Replaced textsize with textbbox
    while draw.textbbox((0,0), n_text, font=n_font)[2] > max_width and getattr(n_font, 'size', 40) > 40:
        n_font = load_font(getattr(n_font, 'size', 86) - 4)
    # Replaced textsize with textbbox
    nw, nh = draw.textbbox((0,0), n_text, font=n_font)[2:]
    draw.text(((W - nw) / 2, 420), n_text, fill=(0,0,0), font=n_font)

    # Course line (wrap if long)
    course_line = f"has successfully completed the course {course}"
    wrapped = textwrap.fill(course_line, width=40)
    # Replaced textsize with textbbox
    cw, ch = draw.textbbox((0,0), wrapped, font=body_font)[2:]
    draw.multiline_text(((W - cw) / 2, 540), wrapped, fill=(55,55,55), font=body_font, align="center")

    # Date & ID
    draw.text((W*0.12, H-220), f"Date: {date_str}", fill=(85,85,85), font=small_font)
    draw.text((W*0.12, H-160), f"Certificate ID: {cert_id}", fill=(85,85,85), font=small_font)

    return im

In [32]:
#@title 4) Generate certificates (PNG + PDF) and ZIP them
import pandas as pd, os, io, zipfile
from datetime import datetime

assert 'excel_path' in globals() and excel_path, "Excel path is not set (run 2A or enable 2B)."
assert 'template_path' in globals() and template_path, "Template path is not set (run 2A or enable 2B)."

df = pd.read_excel(excel_path)

def find_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        for k, v in cols.items():
            if k == cand.lower():
                return v
    for cand in candidates:
        for k, v in cols.items():
            if cand.lower() in k:
                return v
    return None

name_col = find_col(df, ["name","student","student name","full name"])
course_col = find_col(df, ["course","program","training"])
date_col = find_col(df, ["date","issue date","certificate date"])
id_col = find_col(df, ["certificateid","certid","id"])

if not name_col:
    raise SystemExit("Couldn't find a Name column (try Name/Student/Full Name)")
if not course_col:
    print("[WARN] No Course column found. All certificates will show a blank course.")
if not date_col:
    print("[WARN] No Date column found. Using today's date.")

os.makedirs('output', exist_ok=True)

zip_bytes = io.BytesIO()
with zipfile.ZipFile(zip_bytes, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for i, row in df.iterrows():
        name = str(row[name_col]).strip()
        course = str(row[course_col]).strip() if course_col and not pd.isna(row[course_col]) else ""
        dval = str(row[date_col]).split(' ')[0] if date_col and not pd.isna(row[date_col]) else datetime.today().strftime('%Y-%m-%d')
        cert_id = str(row[id_col]).strip() if id_col and not pd.isna(row[id_col]) else f"AUTO-{i+1:04d}"

        im = draw_certificate(template_path, name, course, dval, cert_id)
        safe = "".join(c for c in name if c.isalnum() or c in (' ','-','_')).strip().replace(' ','_')
        png_name = f"{cert_id}_{safe}.png"
        pdf_name = f"{cert_id}_{safe}.pdf"

        # Save to local output folder
        im.save(os.path.join('output', png_name), 'PNG')
        im.save(os.path.join('output', pdf_name), 'PDF')

        # Add to ZIP (in-memory)
        buf_png = io.BytesIO(); im.save(buf_png, 'PNG')
        zf.writestr(png_name, buf_png.getvalue())
        buf_pdf = io.BytesIO(); im.save(buf_pdf, 'PDF')
        zf.writestr(pdf_name, buf_pdf.getvalue())

with open('certificates.zip', 'wb') as f:
    f.write(zip_bytes.getvalue())

print('Done! Files written to ./output and certificates.zip')

Done! Files written to ./output and certificates.zip


In [33]:
#@title 5) Download the ZIP
from google.colab import files
files.download('certificates.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### (Optional) Create sample files to test quickly
Run the cell below to generate a **sample Excel** and a **simple template PNG** if you don't have your own yet.

In [ ]:
#@title Create sample students.xlsx and template.png
import pandas as pd, os
from PIL import Image, ImageDraw

os.makedirs('input', exist_ok=True)
sdf = pd.DataFrame([
    {"CertificateID":"CERT-0001","Name":"Aditi Sharma","Course":"Python Basics","Date":"2025-08-24"},
    {"CertificateID":"CERT-0002","Name":"Rahul Verma","Course":"Python Basics","Date":"2025-08-24"},
])
sdf.to_excel('input/students.xlsx', index=False)

W,H = 1600,1131
bg = Image.new('RGB',(W,H),(248,248,248))
draw = ImageDraw.Draw(bg)
draw.rectangle([40,40,W-40,H-40], outline=(60,60,60), width=6)
draw.text((W*0.33,180),'Certificate of Completion', fill=(20,20,20))
draw.text((W*0.40,320),'This certifies that', fill=(75,75,75))
draw.text((W*0.45,420),'«Name»', fill=(0,0,0))
draw.text((W*0.30,540),'has successfully completed the course «Course»', fill=(55,55,55))
draw.text((W*0.12,H-220),'Date: «Date»', fill=(85,85,85))
draw.text((W*0.12,H-160),'Certificate ID: «CertificateID»', fill=(85,85,85))
bg.save('input/certificate_template.png')
print('Sample files saved in ./input')